# Project 1: Build an LLM Playground

Welcome to your first project! In this project, you'll build a simple large language model (LLM) playground, an interactive environment where you can experiment with LLMs and understand how they work under the hood.

The goal here is to understand the foundations and mechanics behind LLMs rather than relying on higher-level abstractions or frameworks. You'll see what happens “under the hood”, how an LLM receives a text, processes it, and generate a response. In later projects, you'll use frameworks like Ollama and LangChain that simplify many of these steps. But before that, this project will help you build a solid mental model of how LLMs actually work.

We'll use Google Colab, a free browser-based platform that lets you run Python code and machine learning models without installing anything locally. Click the button below to open this notebook in Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bytebyteai/ai-engineering-cohort-3/blob/main/project_1/lm_playground.ipynb)

If you prefer to run the project locally, you can use the provided `env.yaml` file to create a compatible environment using conda. To do so, open a terminal in the same directory as this notebook and run:

```bash
# Create and activate the conda environment
conda env create -f env.yaml && conda activate llm_playground

# Register this environment as a Jupyter kernel
python -m ipykernel install --user --name=llm_playground --display-name "llm_playground"
```


---
## Learning Objectives  
- Understand tokenization and how raw text is converted into a sequence of discrete tokens
- Inspect GPT-2 and the Transformer architecture
- Learn how to load pretrained LLMs using Hugging Face
- Explore decoding strategies to generate text from LLMs
- Compare completion models with instruction-tuned models


Let's get started!

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# ENVIRONMENT SETUP: Verify Required Libraries
#
# Purpose: Check if all necessary libraries are installed and working
# - torch: PyTorch deep learning framework (like TensorFlow)
# - transformers: Hugging Face library for pretrained models
# - tiktoken: Fast tokenization library from OpenAI
#
# Example: If this runs successfully, you'll see version numbers printed
# ═══════════════════════════════════════════════════════════════════
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
print("✅ Environment check complete. You're good to go!")

# 1 - Tokenization

A neural network cannot process raw text directly. It needs numbers.
Tokenization is the process of converting text into numerical IDs that models can understand. In this section, you will learn how tokenization works in practice and why it is an essential step in every language model pipeline.

Tokenization methods generally fall into three main categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word-level tokenization
This method splits text by whitespace and treats each word as a single token. In the next cell, you will implement a basic word-level tokenizer by building a vocabulary that maps words to IDs and writing `encode` and `decode` functions.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# WORD-LEVEL TOKENIZATION: Building Vocabulary
#
# Concept: Create a mapping between words and unique integer IDs
# Process:
#   1. Collect all unique words from your corpus
#   2. Sort them alphabetically for consistency
#   3. Assign each word a unique ID (0, 1, 2, ...)
#
# Example: "the" → 15, "quick" → 13, "fox" → 3
#          word2id = {'the': 15, 'quick': 13, 'fox': 3}
#          id2word = {15: 'the', 13: 'quick', 3: 'fox'}
#
# Note: We need bidirectional mappings for encoding AND decoding
# ═══════════════════════════════════════════════════════════════════

# Creating a tiny corpus.
corpus = [
    "The quick brown fox jumps over the lazy dog",
    "Tokenization converts text to numbers",
    "Large language models predict the next token"
]

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
all_words = " ".join(corpus).lower().split()  # Combine all text, lowercase, split by spaces
vocab = sorted(list(set(all_words)))  # Get unique words and sort them
word2id = {word: i for i, word in enumerate(vocab)}  # Map: word → ID
id2word = {i: word for i, word in enumerate(vocab)}  # Map: ID → word

print(f"Vocabulary size: {len(vocab)} words")
print("First 15 vocab entries:", vocab[:150])
print("words to id:", word2id)
print("id 2 words:", id2word)


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# WORD-LEVEL TOKENIZATION: Encode and Decode Functions
#
# encode(): Converts human text → machine numbers (IDs)
#   Example: "The quick fox" → [15, 13, 3]
#
# decode(): Converts machine numbers → human text
#   Example: [15, 13, 3] → "the quick fox"
#
# Out-of-Vocabulary (OOV): Words not in vocab get ID -1 and decode to [UNK]
#   Example: "fast" not in vocab → -1 → "[UNK]"
# ═══════════════════════════════════════════════════════════════════

# Step 2: Define encode and decode functions
def encode(text):
    # converts text to token IDs
    # .get(word.lower(), -1) returns -1 if word not found in vocabulary
    return [word2id.get(word.lower(), -1) for word in text.split()]

def decode(ids):
    # converts token IDs back to text
    # .get(i, "[UNK]") returns "[UNK]" if ID not found
    return " ".join([id2word.get(i, "[UNK]") for i in ids])


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TESTING WORD-LEVEL TOKENIZATION
#
# Test 1: In-vocabulary words (should encode/decode perfectly)
#   "The quick brown fox" → [15, 13, 0, 3] → "the quick brown fox"
#
# Test 2: Out-of-vocabulary (OOV) words (should show [UNK] for unknown)
#   "The fast green bird" → [15, -1, -1, -1] → "the [UNK] [UNK] [UNK]"
#   ("fast", "green", "bird" were never in our training corpus)
# ═══════════════════════════════════════════════════════════════════

# Step 3: Test your tokenizer with random sentences.
test_sentence = "The quick brown fox"
encoded = encode(test_sentence)
decoded = decode(encoded)
print(f"Original: {test_sentence}")
print(f"Encoded:  {encoded}")
print(f"Decoded:  {decoded}")

test_oov = "The fast green bird"
print(f"\nOOV Test: {test_oov}")
print(f"Encoded:  {encode(test_oov)}")
print(f"Decoded:  {decode(encode(test_oov))}")


While word-level tokenization is simple and easy to understand, it has two key limitations that make it impractical for large-scale models:
1.  large vocabulary size: every new word or variation (for example, run, runs, running) increases the total vocabulary, leading to higher memory and training costs.
2. Out-of-vocabulary (OOV) problem: the model cannot handle unseen or rare words that were not part of the training vocabulary, so they must be replaced with a generic [UNK] token.

The next section introduces character-level tokenization, where text is represented as individual characters instead of words.

### 1.2 - Character-level tokenization

In this approach, every single character (including spaces, punctuation, and even emojis) is assigned its own ID.

In the next section, we will rebuild a tokenizer using the same corpus as before, but this time with a character-level approach.
For simplicity, assume we are only using lowercase and uppercase English letters (a-z, A-Z).

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# CHARACTER-LEVEL TOKENIZATION: Building Vocabulary
#
# Concept: Instead of words, treat each CHARACTER as a token
# Vocabulary: All uppercase (A-Z) + lowercase (a-z) + space + [UNK]
#   Total: 52 letters + 2 special = 54 tokens
#
# Example:
#   'H' → ID 33
#   'e' → ID 4
#   'l' → ID 11
#   'l' → ID 11
#   'o' → ID 14
#
# Advantage: No out-of-vocabulary problem (every letter is known)
# Disadvantage: Words become much longer sequences
# ═══════════════════════════════════════════════════════════════════

import string

# Step 1: Create a vocabulary that includes all uppercase and lowercase letters.
vocab = list(string.ascii_letters) + [" ", "[UNK]"]  # a-z, A-Z, space, unknown
char2id = {char: i for i, char in enumerate(vocab)}
id2char = {i: char for i, char in enumerate(vocab)}

print(f"Vocabulary size: {len(vocab)} (52 letters + 2 specials)")


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# CHARACTER-LEVEL: Encode and Decode Functions
#
# encode(): Converts each CHARACTER to its ID
#   Example: "Hi" → [33, 8] (H=33, i=8)
#
# decode(): Converts each ID back to its CHARACTER
#   Example: [33, 8] → "Hi"
#
# Note: Unknown characters (like numbers, emojis) map to [UNK]
#   Example: "Hi2" → [33, 8, UNK_ID] (if '2' not in vocab)
# ═══════════════════════════════════════════════════════════════════

# Step 2: Implement encode() and decode() functions to convert between text and IDs.
def encode(text):
    # convert text to list of IDs
    # Each character is looked up individually
    return [char2id.get(c, char2id["[UNK]"]) for c in text]

def decode(ids):
    # Convert list of IDs to text
    # Join characters back together (no spaces between letters)
    return "".join([id2char.get(i, "[UNK]") for i in ids])


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TESTING CHARACTER-LEVEL TOKENIZATION
#
# Example: "Hello World" (11 characters)
#   Encoded: [33, 4, 11, 11, 14, 52, 48, 14, 17, 11, 3]
#            H   e  l   l   o   <sp> W   o   r   l   d
#
# Notice: Much longer sequence than word-level tokenization!
#   Word-level: "Hello World" → 2 tokens ["Hello", "World"]
#   Char-level: "Hello World" → 11 tokens (each character)
# ═══════════════════════════════════════════════════════════════════

# Step 3: Test your tokenizer on a short sample word.
sample = "Hello World"
encoded = encode(sample)
decoded = decode(encoded)
print(f"Sample: {sample}")
print(f"Encoded: {encoded}")
print(f"Decoded: {decoded}")


Character-level tokenization solves the out-of-vocabulary problem but introduces new challenges:

1. Longer sequences: because each word becomes many tokens, models need to process much longer inputs.
2. Weaker semantic representation: individual characters carry very little meaning, so models must learn relationships across many steps.
3. Higher computational cost: longer sequences lead to more tokens per input, which increases training and inference time.

To find a better balance between vocabulary size and sequence length, we move to subword-level tokenization next.

### 1.3 - Subword-level tokenization

Sub-word methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** common groups of characters and merge them into tokens. For example, the word **unbelievable** might turn into three tokens: **["un", "believ", "able"]**. This approach strikes a balance between word-level and character-level methods and fix their limitations.

The BPE algorithm builds a vocabulary iteratively using the following process:
1. Start with individual characters (each character is a token).
2. Count all adjacent pairs of tokens in a large text corpus.
3. Merge the most frequent pair into a new token.

Repeat steps 2 and 3 until you reach the desired vocabulary size (for example, 50,000 tokens).

In the next cell, you will experiment with BPE in practice to see how it compresses text into meaningful subword units. Instead of implementing the algorithm from scratch, you will use a pretrained tokenizer, which was already trained on a large text corpus to build its vocabulary, such as the data used to train `GPT-2`. This allows you to see how BPE works in practice with a real, learned vocabulary.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# SUBWORD (BPE) TOKENIZATION: Loading Pretrained Tokenizer
#
# Concept: Byte-Pair Encoding (BPE) merges common character pairs
# Process:
#   1. Start with all characters (like character-level)
#   2. Find most frequent pairs: e.g., 't'+'h' → 'th'
#   3. Merge them into single tokens
#   4. Repeat until vocabulary reaches desired size (e.g., 50K tokens)
#
# Example: "unbelievable" might become ["un", "believ", "able"]
#   - Not full word (word-level would be: ["unbelievable"])
#   - Not each character (char-level would be 12 tokens)
#   - Perfect middle ground: 3 meaningful subword tokens
#
# Note: Using pretrained tokenizer from GPT model (already learned merges)
# ═══════════════════════════════════════════════════════════════════

from transformers import AutoTokenizer

# Step 1: Load a pretrained GPT-2 tokenizer from Hugging Face.
# This tokenizer was trained on massive text corpus to learn optimal merges
tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# BPE: Wrapper Functions for Encode/Decode
#
# These wrap the pretrained tokenizer's methods for consistency
# The tokenizer handles all the complex BPE logic internally
#
# Example workflow:
#   encode("Hello") → tokenizer looks up learned merges → [15496]
#   decode([15496]) → tokenizer reverses lookup → "Hello"
# ═══════════════════════════════════════════════════════════════════

# Step 2: Use it to write encode and decode helper functions.
def encode(text):
    return tokenizer.encode(text)

def decode(ids):
    return tokenizer.decode(ids)


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# BPE: Inspecting Token Breakdown
#
# tokenize(): Shows HOW the text was split into subword pieces
# convert_tokens_to_ids(): Converts subwords to their integer IDs
#
# Example: "Unbelievable" →
#   Tokens: ['Un', 'bel', 'ievable']  (3 subwords)
#   IDs: [2265, 9880, 45794]  (their vocabulary IDs)
#
# Special prefix 'Ġ': Indicates space before word
#   'Ġtoken' means " token" (with leading space)
#
# Comparison:
#   Word-level: Would need "Unbelievable" in vocab (huge vocab)
#   Char-level: 13 separate tokens (U-n-b-e-l-i-e-v-a-b-l-e)
#   BPE: 3 meaningful subwords (best of both worlds)
# ═══════════════════════════════════════════════════════════════════

# 3. Inspect the tokens to see how BPE breaks words apart.
sample = "Unbelievable tokenization powers!"
tokens = tokenizer.tokenize(sample)  # Break into subword pieces
ids = tokenizer.convert_tokens_to_ids(tokens)  # Convert to IDs
print(f"Sample: {sample}")
print(f"Tokens: {tokens}")  # See the subword breakdown
print(f"IDs:    {ids}")
print(f"IDs to Tokens:", decode(ids))
print(f"IDs to Tokens:", decode([2232,232,23,23,35345,6754,4, 100010]))  # Random IDs for comparison


### 1.4 - TikToken

`tiktoken` is a fast, production-ready library for tokenization used by OpenAI models.
It is designed for efficiency and consistency with how OpenAI counts tokens in GPT models.

In this section, you will explore how different model families use different tokenizers. We will compare tokenizers used to train `GPT-2` and more powerful models such as `GPT-4`. By trying both, you will see how tokenization has evolved to handle more diverse text (including emojis, Unicode, and special characters) while remaining efficient.

In the next cell, you will use tiktoken to load these encodings and inspect how each one splits the same text. You may find reading this doc helpful: https://github.com/openai/tiktoken

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TIKTOKEN: Comparing GPT-2 vs GPT-4 Tokenizers
#
# Why different tokenizers?
#   - GPT-2 (2019): Older, trained on mostly English text
#   - GPT-4 (2023): Newer, trained on diverse multilingual data + emojis
#
# Example: "🌟" (star emoji)
#   GPT-2: Breaks into multiple weird tokens (3 tokens: � � �)
#   GPT-4: Single clean token (1 token)
#
# Why this matters:
#   - Fewer tokens = faster processing, lower costs
#   - Better tokenization = better understanding
#   - GPT-4's tokenizer handles code, unicode, emojis much better
#
# Token count comparison for same text:
#   "The 🌟 star-programmer implemented AGI overnight."
#   GPT-2: 13 tokens (emoji causes issues)
#   GPT-4: 12 tokens (more efficient)
# ═══════════════════════════════════════════════════════════════════

import tiktoken

# Compare GPT-2 and GPT-4 tokenizers using tiktoken.

# Step 1: Load two tokenizers
enc_gpt2 = tiktoken.get_encoding("gpt2")  # Older tokenizer
enc_gpt4 = tiktoken.get_encoding("cl100k_base")  # GPT-4's tokenizer

# Step 2: Encode the same sentence with both and observe how they differ
sentence = "The 🌟 star-programmer implemented AGI overnight."
ids_gpt2 = enc_gpt2.encode(sentence)
ids_gpt4 = enc_gpt4.encode(sentence)

# Decode each token individually to see the breakdown
print(f"GPT-2 tokens ({len(ids_gpt2)}): {[enc_gpt2.decode([i]) for i in ids_gpt2]}")
print(f"GPT-4 tokens ({len(ids_gpt4)}): {[enc_gpt4.decode([i]) for i in ids_gpt4]}")


Try changing the input sentence and observe how different tokenizers behave.
Experiment with:
- Emojis, special characters, or punctuation
- Code snippets or structured text
- Non-English text (for example, Japanese, French, or Arabic)

If you are curious, you can also attempt to implement the BPE algorithm yourself using a small text corpus to see how token merges are learned in practice.

### 1.5 - Key Takeaways
- **Word-level**: simple and intuitive, but limited by large vocabularies and out-of-vocabulary issues
- **Character-level**: flexible and covers all text, but produces long sequences that are harder to model
- **Subword / BPE**: balances both worlds and is the default choice for most modern LLMs
- **TikToken**: a production-ready tokenizer used in OpenAI models, demonstrating how optimized subword vocabularies are applied in real systems

# 2. What is a Language Model?

At its core, a **language model (LM)** is just a *very large* mathematical function built from many neural-network layers.  
Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it learns to output a probability for the next token `tₙ₊₁`.


Each layer performs basic mathematical operations such as matrix multiplication and attention. When hundreds of these layers are stacked together, the model learns complex patterns and statistical relationships in text. The final output is a vector of scores that represents how likely each possible token is to appear next. You can think of the entire model as one giant equation whose parameters were optimized during training to minimize prediction errors.

### 2.1 - A Single `Linear` Layer

Before jumping into Transformers, let's start with the simplest building block: a `Linear` layer.

A Linear layer computes `y = Wx + b`.

Where:  
  * `x` - input vector  
  * `W` - weight matrix (learned)  
  * `b` - bias vector (learned)

Although this operation looks simple, stacking many linear layers (along with nonlinear activation functions) allows neural networks to model highly complex relationships in data.

In the next cell, you will explore how a **Linear layer** works in practice by implementing one from scratch. You will define the weights and bias, then perform the matrix multiplication and addition manually to see what happens inside this layer. You may find the following links useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html
- https://docs.pytorch.org/docs/stable/generated/torch.randn.html
- https://docs.pytorch.org/docs/stable/generated/torch.matmul.html

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# LINEAR LAYER: Custom Implementation
#
# Concept: A linear layer performs: y = Wx + b
#   - x: input vector (e.g., [1.0, -1.0, 0.5])
#   - W: weight matrix (learned during training)
#   - b: bias vector (learned during training)
#   - y: output vector
#
# Example calculation:
#   Input x:  [1.0, -1.0, 0.5]
#   Weight W: [[0.78, -0.32, -0.02],  ← produces output[0]
#              [-0.21, -2.41, -0.47]] ← produces output[1]
#   Bias b:   [0.78, 0.40]
#
#   Output y[0] = (0.78*1.0) + (-0.32*-1.0) + (-0.02*0.5) + 0.78 = 1.87
#   Output y[1] = (-0.21*1.0) + (-2.41*-1.0) + (-0.47*0.5) + 0.40 = 2.36
#
# Why important: Neural networks are STACKS of these transformations
#   Input → Linear → ReLU → Linear → ReLU → ... → Output
# ═══════════════════════════════════════════════════════════════════

import torch
import torch.nn as nn

# Define a MyLinear PyTorch module and perform y = Wx + b.

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(MyLinear, self).__init__()
        # Initialize weights and bias with random values
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        # Perform matrix multiplication + bias
        return torch.matmul(self.weight, x) + self.bias

# Create a linear layer: 3 inputs → 2 outputs
lin = MyLinear(3, 2)
x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)
print("Bias   :", lin.bias)
print("Output :", lin(x))


Next, you will use PyTorch's built-in nn.Linear module, which performs the same computation `(y = Wx + b)` but automatically handles parameter initialization, gradient tracking, and integration with the rest of a neural network. Comparing your manual implementation with this built-in version will help you understand what a linear layer does and how deep learning frameworks make these operations easier to use.

You may find this link useful:
- https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# LINEAR LAYER: Using PyTorch Built-in
#
# PyTorch's nn.Linear does the EXACT SAME thing as our MyLinear
# But it's optimized and includes extra features:
#   - Better initialization strategies
#   - Automatic gradient tracking for backpropagation
#   - GPU acceleration support
#   - Memory efficiency optimizations
#
# Comparison:
#   Our MyLinear:      Good for learning concepts
#   PyTorch nn.Linear: Use this in real projects
#
# Both do: y = Wx + b, but nn.Linear is production-ready
# ═══════════════════════════════════════════════════════════════════

import torch.nn as nn, torch

# Create a linear layer using pytorch's nn.Linear
lin = nn.Linear(3, 2)  # 3 input features → 2 output features

x = torch.tensor([1.0, -1.0, 0.5])
print("Input :", x)
print("Weights:", lin.weight)  # Automatically initialized
print("Bias   :", lin.bias)    # Automatically initialized
print("Output :", lin(x))      # Forward pass


### 2.2 - A `Transformer` Layer

Most LLMs are a **stack of identical Transformer blocks**. Each block fuses two main components:

| Step | What it does | Where it lives in code |
|------|--------------|------------------------|
| **Multi-Head Self-Attention** | Every token looks at every other token and decides *what matters*. | `block.attn` |
| **Feed-Forward Network (MLP)** | Re-mixes information token-by-token. | `block.mlp` |

In the next section, you will load `GPT-2` and inspect its first Transformer block to see these components in a real model. You will locate its layers, print their shapes and parameters, and understand how a block processes a batch of token embeddings.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TRANSFORMER ARCHITECTURE: Loading and Inspecting GPT-2
#
# GPT-2 Structure:
#   - 12 identical Transformer blocks stacked on top of each other
#   - Each block has ~10 million parameters
#   - Total: ~124 million parameters
#
# What's in a Transformer Block?
#   1. Multi-Head Attention (attn): Lets tokens "look at" other tokens
#      → "The cat sat" - "sat" can attend to "cat" to understand subject
#
#   2. Feed-Forward Network (mlp): Processes each token independently
#      → Applies: Linear → ReLU → Linear transformations
#
#   3. Layer Normalization (ln_1, ln_2): Stabilizes training
#      → Keeps values in reasonable range
#
# Example flow:
#   Token embeddings → Block 0 → Block 1 → ... → Block 11 → Output
# ═══════════════════════════════════════════════════════════════════

import torch
from transformers import GPT2LMHeadModel

# Step 1: load the smallest GPT-2 model
# This downloads ~500MB from Hugging Face (cached for future use)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Step 2: # Inspect the first Transformer block one by printing it.
# You'll see: LayerNorm (ln_1, ln_2), Attention (attn), MLP (mlp)
print(model.transformer.h[0])
print(model)


In this section, you will run a minimal forward pass through one GPT-2 block to understand how tokens are transformed inside the model.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# FORWARD PASS: Seeing How a Transformer Block Works
#
# Process:
#   1. Generate random token IDs (simulating real input)
#      Example: [42, 1523, 8, 99, 234, 7, 1000, 45] (8 random tokens)
#
#   2. Convert to embeddings (numbers the model can process)
#      - wte: Token embeddings (what each word means)
#      - wpe: Position embeddings (where each word is in sequence)
#      Example: Each token becomes a 768-dimensional vector
#
#   3. Pass through one Transformer block
#      Input shape:  [1, 8, 768]  (batch=1, seq_len=8, embed_dim=768)
#      Output shape: [1, 8, 768]  (same shape, but values transformed)
#
# Key insight: Transformer blocks maintain sequence shape
#   They transform the MEANING but keep dimensions constant
# ═══════════════════════════════════════════════════════════════════

# Step 1: Create a small dummy input with a sequence of 8 random token IDs.
batch_size, seq_len = 1, 8
input_ids = torch.randint(0, model.config.vocab_size, (batch_size, seq_len))

# Step 2: Convert token IDs into embeddings
wte = model.transformer.wte(input_ids)  # Token embeddings
wpe = model.transformer.wpe(torch.arange(seq_len).unsqueeze(0))  # Position embeddings
hidden_states = wte + wpe  # Combine both types of information

# Step 3: Pass the embeddings through a single Transformer block
block_output = model.transformer.h[0](hidden_states)[0]

# Step 4: Inspect the result
print(f"Block output shape: {block_output.shape}")  # Should be [1, 8, 768]


### 2.3 - Inside GPT-2

GPT-2 is essentially a stack of identical Transformer blocks arranged in sequence.
Each block contains attention, feed-forward, and normalization layers that process token representations step by step.

In this section, you will print the modules inside the GPT-2 Transformer to see how these components are organized.
This will help you understand how the model scales from a single block to a full network of many layers working together.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# MODEL STRUCTURE: Listing All Components
#
# This shows the high-level architecture of GPT-2's transformer
#
# Expected output:
#   - wte: Word/Token embeddings (vocab_size → 768 dimensions)
#   - wpe: Position embeddings (max_position → 768 dimensions)
#   - drop: Dropout layer (regularization during training)
#   - h: ModuleList containing all 12 Transformer blocks
#   - ln_f: Final layer normalization before output
#
# Think of it as: Input → Embeddings → 12 Blocks → Normalize → Output
# ═══════════════════════════════════════════════════════════════════

# Print the name of all layers inside gpt.transformer.
for name, child in model.transformer.named_children():
    print(name)


As you can see, the Transformer holds various modules, arranged from a list of blocks (`h`). The following table summarizes these modules:

| Step | What it does | Why it matters |
|------|--------------|----------------|
| **Token → Embedding** | Converts IDs to vectors | Gives the model a numeric “handle” on words |
| **Positional Encoding** | Adds “where am I?” info | Order matters in language |
| **Multi-Head Self-Attention** | Each token asks “which other tokens should I look at?” | Lets the model relate words across a sentence |
| **Feed-Forward Network** | Two stacked Linear layers with a non-linearity | Mixes information and adds depth |
| **LayerNorm & Residual** | Stabilize training and help gradients flow | Keeps very deep networks trainable |


### 2.4 LLM's output

When you pass a sequence of tokens through a language model, it produces a tensor of logits with shape
`(batch_size, seq_len, vocab_size)`.
Each position in the sequence receives a vector of scores representing how likely every possible token is to appear next. By applying a softmax function on the last dimension, these logits can be converted into probabilities that sum to 1.

In the next cell, you will feed an 8-token dummy sequence into GPT-2, print the shape of its logits, and display the five most likely next tokens predicted for the final position in the sequence.


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# LLM OUTPUT: Setting Up for Next Token Prediction
#
# Goal: See how a language model predicts the NEXT token
#
# Setup:
#   - GPT2TokenizerFast: Fast version of tokenizer for encoding text
#   - GPT2LMHeadModel: Complete GPT-2 with prediction "head"
#     (The "LMHead" is a final linear layer: hidden_states → vocab_size)
#
# Workflow:
#   Text → Tokenizer → IDs → Model → Logits → Probabilities → Prediction
#   "Hello" → [15496] → GPT-2 → [50257 scores] → softmax → "world" (highest)
# ═══════════════════════════════════════════════════════════════════

import torch, torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Step 1: Load GPT-2 model and its tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TOKENIZATION: Converting Text to Model Input
#
# return_tensors="pt": Returns PyTorch tensors (not lists)
#
# Example:
#   Input text: "Hello my name"
#   Tokenized: {"input_ids": tensor([[15496, 616, 1438]])}}
#              token IDs for: "Hello", "Ġmy", "Ġname"
#
# This format is ready to feed directly into the model
# ═══════════════════════════════════════════════════════════════════

# Step 2: Tokenize input text
text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")  # Convert to PyTorch format


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# MODEL FORWARD PASS: Getting Predictions
#
# torch.no_grad(): Disable gradient calculation (we're not training)
#   → Saves memory and speeds up inference
#
# model(**inputs): Run the full forward pass
#   Input: ["Hello", "my", "name"] (3 tokens)
#   Output: logits tensor of shape [1, 3, 50257]
#     - 1: batch size
#     - 3: sequence length (one prediction per input token)
#     - 50257: vocabulary size (score for EVERY possible next token)
#
# Logits: Raw scores (not probabilities yet)
#   Higher score = more likely next token
#   Example logits for position "name": [0.5, -2.3, 1.8, ...] (50257 values)
# ═══════════════════════════════════════════════════════════════════

# Step 2: Prepare input text and tokenize it
# This was the missing step in the original snippet.
text = "Hello my name"
inputs = tokenizer(text, return_tensors="pt")

# Step 3: Pass the input IDs to the model
with torch.no_grad():  # No gradient tracking needed for inference
    outputs = model(**inputs)
    logits = outputs.logits  # Get raw prediction scores


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# NEXT TOKEN PREDICTION: From Logits to Words
#
# Process:
#   1. Extract logits for LAST token: "name" → [50257 scores]
#   2. Apply softmax: Converts scores to probabilities (sum = 1.0)
#      Example: [0.5, -2.3, 1.8] → [0.23, 0.01, 0.76]
#   3. Find top 5 most probable tokens
#   4. Decode IDs back to readable text
#
# Example output:
#   Input: "Hello my name"
#   Top predictions:
#     1. " is" (prob: 0.4521)  " Hello my name is"
#     2. "'s" (prob: 0.1234)   "Hello my name's"
#     3. " was" (prob: 0.0876)   "Hello my name was"
#
# This is the CORE of how language models work:
#   Given context → Predict most likely next token → Repeat
# ═══════════════════════════════════════════════════════════════════

# Step 4: Predict the next token
last_token_logits = logits[0, -1, :]  # Get scores for position after "name"
probs = F.softmax(last_token_logits, dim=-1)  # Convert to probabilities
top_probs, top_indices = torch.topk(probs, 5)  # Get top 5 predictions

# Print the most likely next tokens
for i in range(5):
    token = tokenizer.decode([top_indices[i]])
    print(f"Top {i+1}: '{token}' (prob: {top_probs[i].item():.4f})")



In [ ]:
# Alternate full code

import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# =========================================================================
# STEP 1: LOAD MODEL AND TOKENIZER
# =========================================================================
# The tokenizer converts text into numbers. GPT2TokenizerFast uses Byte-Pair Encoding (BPE).
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# The model is the neural network itself. GPT2LMHeadModel includes the 
# transformer blocks + the "Language Modeling Head" which predicts the next token.
model = GPT2LMHeadModel.from_pretrained("gpt2")

# =========================================================================
# STEP 2: TOKENIZATION (TEXT TO NUMBERS)
# =========================================================================
text = "The future of artificial intelligence is"

# tokenizer() converts the string into a dictionary:
# - 'input_ids': The integer ID for each word/sub-word token.
# - 'attention_mask': Tells the model which tokens to focus on (useful for padding).
# return_tensors="pt" ensures the output is a PyTorch Tensor, not a Python list.
inputs = tokenizer(text, return_tensors="pt")

# =========================================================================
# STEP 3: MODEL FORWARD PASS
# =========================================================================
# torch.no_grad() disables gradient calculation to save memory and speed up 
# inference since we aren't training (updating weights).
with torch.no_grad():
    # **inputs "unpacks" the dictionary (input_ids=..., attention_mask=...)
    outputs = model(**inputs)
    
    # Logits are the raw, unnormalized scores from the last layer of the model.
    # Shape: [batch_size (1), sequence_length (7), vocab_size (50257)]
    logits = outputs.logits

# =========================================================================
# STEP 4: PREDICT THE NEXT TOKEN
# =========================================================================
# We only care about the logits for the VERY LAST token in the sequence 
# because that's the one that predicts what comes next.
# [:, -1, :] means [all batches, last token in sequence, all vocab scores]
next_token_logits = logits[:, -1, :]

# torch.argmax finds the index (ID) with the highest score in the vocabulary.
next_token_id = torch.argmax(next_token_logits, dim=-1)

# decode() converts the integer ID back into a human-readable string.
next_token = tokenizer.decode(next_token_id)

# =========================================================================
# RESULTS
# =========================================================================
print(f"Input text: {text}")
print(f"Token IDs: {inputs['input_ids']}")
print(f"Predicted next token: {next_token}")

# Step 6 Predict the next token
last_token_logits = logits[0, -1, :]  # Get scores for position after "name"
probs = F.softmax(last_token_logits, dim=-1)  # Convert to probabilities
top_probs, top_indices = torch.topk(probs, 5)  # Get top 5 predictions
# Print the most likely next tokens
for i in range(5):
    token = tokenizer.decode([top_indices[i]])
    print(f"Top {i+1}: '{token}' (prob: {top_probs[i].item():.4f})")



### 2.5 - Key Takeaway

A language model is not a black box or something mysterious.
It is a large composition of simple, understandable layers such as linear layers, attention, and normalization, trained together to predict the next token in a sequence.

By learning this next-token prediction task at scale, the model gradually develops an internal understanding of language structure, meaning, and context, which allows it to generate coherent and relevant text.

# 3 - Text Generation (Decoding)
Once a language model has been trained to predict token probabilities, we can use it to generate text.
This process is called text generation or decoding.

At each step, the model outputs a probability distribution over possible next tokens.
A decoding algorithm then selects one token based on that distribution, appends it to the sequence, and repeats the process to build text word by word. Different decoding strategies control how the model chooses the next token and how creative or deterministic the output will be. For example:
- **Greedy** decoding: always pick the token with the highest probability. Simple and consistent, but often repetitive.
- **Top-k** or **Nucleus** (top-p) sampling: randomly sample from the top few likely tokens to add variety.
- Beam search: explores multiple candidate continuations and keeps the best overall sequence.

Note: `Temperature` adjusts randomness in sampling. Higher values make outputs more diverse, while lower values make them more focused and deterministic.

### 3.1 - Greedy decoding
In this section, you will use GPT-2 and Hugging Face's built-in generate method to produce text using the greedy decoding strategy.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TEXT GENERATION: Greedy Decoding Strategy
#
# Greedy = Always pick highest probability token
#
# Algorithm:
#   1. Start with prompt: "Once upon a time"
#   2. Get predictions, pick top token: "there"
#   3. New prompt: "Once upon a time there"
#   4. Repeat until max_new_tokens reached
#
# Example:
#   "Once upon a" → model predicts "time" (90% prob)
#   "Once upon a time" → model predicts "there" (85% prob)
#   "Once upon a time there" → model predicts "was" (88% prob)
#   ... continues for max_new_tokens iterations
#
# Pros: Deterministic (same input = same output), fast
# Cons: Repetitive, boring, can get stuck in loops
#
# do_sample=False: Use greedy (no randomness)
# ═══════════════════════════════════════════════════════════════════

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

# Step 1. Load GPT-2 model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

# Step 2. Implement a text generation function
def generate(model, tokenizer, prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # do_sample=False → use greedy decoding (always pick highest prob)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TESTING GREEDY DECODING
#
# Testing 3 different prompt types:
#   1. Story beginning: "Once upon a time"
#   2. Question: "What is 2+2?"
#   3. Creative prompt: "Suggest a party theme."
#
# Notice: GPT-2 is a COMPLETION model, not instruction-tuned
#   - It continues text, doesn't answer questions directly
#   - "What is 2+2?" might generate: "What is 2+2? I don't know..."
#     (continuing as if writing text, not answering)
# ═══════════════════════════════════════════════════════════════════

tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n GPT-2 | Greedy")
    print(generate(model, tokenizer, prompt, 80))


Naively selecting the single most probable token at each step (known as greedy decoding) often leads to poor results in practice:
- Repetition loops: phrases like “The cat is is is…”
- Short-sighted choices: the most likely token right now might lead to incoherent text later

These issues are why more advanced decoding methods such as top-k and nucleus sampling are commonly used to make model outputs more diverse and natural.

### 3.2 - Top-k and top-p sampling
The generate function you implemented earlier can easily be extended to use different decoding strategies.

In this section, you will reimplement the same function but adapt it to support Top-k and Top-p (nucleus) sampling. These methods introduce controlled randomness, allowing the model to explore multiple plausible continuations instead of always choosing the single most likely next token.

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# ADVANCED DECODING: Greedy, Top-K, and Top-P Sampling
#
# Concept: Introduce randomness to make generation more "creative"
#
# 1. Greedy (top_k=0, top_p=1.0, do_sample=False):
#    Always picks the absolute highest probability. Safe but repetitive.
#
# 2. Top-K Sampling (top_k=50):
#    Filters to the top 50 most likely tokens, then samples from them.
#    Throws away the "long tail" of low-probability, nonsensical tokens.
#
# 3. Top-P (Nucleus) Sampling (top_p=0.9):
#    Samples from the smallest set of tokens whose cumulative probability > 0.9.
#    Dynamic: might sample from 2 tokens (if they're very likely) or 100.
#
# Example: If predicting after "The sky is..."
#   - Greedy: Always "blue"
#   - Top-K/P: Might choose "blue", "cloudy", or "clear"
# ═══════════════════════════════════════════════════════════════════

# Implement `generate` to support 3 strategies: greedy, top_k, and top_p
def generate(model, tokenizer, prompt, strategy="greedy", max_new_tokens=128, temperature=1.0):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    if strategy == "greedy":
        # Deterministic: picks most likely token
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    elif strategy == "top_k":
        # Random selection from top 50 likely candidates
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens, 
            do_sample=True, 
            top_k=50, 
            temperature=temperature
        )
    elif strategy == "top_p":
        # Random selection from candidates forming 90% of probability mass
        outputs = model.generate(
            **inputs, 
            max_new_tokens=max_new_tokens, 
            do_sample=True, 
            top_p=0.9, 
            temperature=temperature
        )
    else:
        # Default fallback
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
        
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TESTING DECODING STRATEGIES
#
# Testing 3 different prompt types:
#   1. Story beginning: "Once upon a time"
#   2. Question: "What is 2+2?"
#   3. Creative prompt: "Suggest a party theme."
#
# Notice: GPT-2 is a COMPLETION model, not instruction-tuned
#   - It continues text, doesn't answer questions directly
#   - Higher temperature = More "wild" or varied completions
# ═══════════════════════════════════════════════════════════════════

tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]

for prompt in tests:
    print(f"\n" + "="*50)
    print(f"PROMPT: {prompt}")
    print("="*50)
    
    # 1. Greedy (Stable/Predictable)
    print(f"\n[Strategy: Greedy]")
    print(generate(model, tokenizer, prompt, "greedy", 50))
    
    # 2. Top-K with Moderate Temperature (Balanced)
    print(f"\n[Strategy: Top-K | Temp: 0.7]")
    print(generate(model, tokenizer, prompt, "top_k", 50, temperature=0.7))
    
    # 3. Top-P with High Temperature (Creative/Chaotic)
    print(f"\n[Strategy: Top-P | Temp: 1.5]")
    print(generate(model, tokenizer, prompt, "top_p", 50, temperature=1.5))


### 3.3 - Try It Yourself

Now it’s time to experiment with text generation. Replace the sample prompts with your own prompts or adjust the decoding strategy.
You can experiment with:
- strategy: "greedy", "beam", "top_k", "top_p"
- temperature: values between 0.2 and 2.0
- k or p: thresholds that control sampling diversity

Try generating the same prompt with `greedy` and `top_p` (for example, 0.9). Notice how even small temperature changes can make the output more focused or more free-form.




# 4 - Completion vs. Instruction-tuned LLMs

So far, we have used `GPT-2` to generate text from a given input prompt. However, `GPT-2` is just a completion model. It simply continues the provided text without understanding it as a task or question. It is not designed to engage in dialogue or follow instructions.

In contrast, instruction-tuned LLMs (such as `Qwen-Chat`) undergo an additional post-training stage after base pre-training. This process fine-tunes the model to behave helpfully and safely when interacting with users. Because of this extra stage, instruction-tuned models can:

- Interpret prompts as requests rather than just text to continue
- Stay in conversation mode, answering questions and following steps
- Handle refusals and safety boundaries appropriately
- Maintain a consistent helpful persona, rather than drifting into storytelling

### 4.1 - `Qwen/Qwen3-0.6B` vs. `GPT2`

In the next cell, you will feed the same prompt to two different models:

- GPT-2 (completion-only): continues the text in the same writing style
- Qwen/Qwen3-0.6B (instruction-tuned): interprets the input as an instruction and responds helpfully

Comparing the two outputs will make the difference between completion and instruction-tuned behavior clear.



In [ ]:
# ═══════════════════════════════════════════════════════════════════
# MODEL COMPARISON: Base Model vs Instruction-Tuned Model
#
# Concept: Pre-training vs. Post-training
#
# 1. GPT-2 (Base): Trained to "complete the next word" from the internet.
#    - Behavior: Predicts what comes next (storytelling, document completion)
#    - Response to "What is 2+2?": Might write more questions or random text
#
# 2. Qwen-Chat (Instruct): Base model + Fine-tuning for conversation
#    - Behavior: Follows directions, answers questions, acts as assistant
#    - Response to "What is 2+2?": "2 + 2 is 4."
#
# Loading two separate tokenizers and models for direct comparison.
# ═══════════════════════════════════════════════════════════════════

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-2 (Smallest variant, purely completion-based)
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

# Load Qwen-2.5-0.5B-Instruct (Small but very capable, instruction-tuned)
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct").to(device)


We have now downloaded two small checkpoints: GPT-2 (124M parameters) and Qwen3-0.6B (600M parameters). If the previous cell took some time to run, that was mainly due to model download speed. The models will be cached locally, so future runs will be faster.

Next, we will generate text using our generate function with both models and the same prompt to directly compare how a completion-only model (GPT-2) behaves differently from an instruction-tuned model (Qwen).

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# EXECUTING COMPARISON: The "AHA!" Moment
#
# We run the same prompts through both models to see the DRASTIC difference.
#
# Example Expected Difference:
#   Prompt: "Suggest a party theme."
#   GPT-2: "Suggest a party theme. The first one is a masquerade..." (continues list)
#   Qwen: "Sure! Here are some fun party themes: 1. Tropical Luau, 2. Retro 80s..."
#
# Key insight: Qwen understands the INTENT to get a suggestion, GPT-2 just sees text.
# ═══════════════════════════════════════════════════════════════════

tests=[("Once upon a time", "greedy"),("What is 2+2?", "top_k"),("Suggest a party theme.", "top_p")]

for prompt, strategy in tests:
    # Run through GPT-2
    print(f"\nModel: GPT-2 | Strategy: {strategy}")
    print(generate(gpt2_model, gpt2_tokenizer, prompt, strategy, 40))
    
    # Run through Qwen
    print(f"\nModel: Qwen | Strategy: {strategy}")
    # Note: strategy.replace('_','-') handles "top-p" vs "top_p" for the function logic if needed
    print(generate(qwen_model, qwen_tokenizer, prompt, strategy.replace('_','-'), 40))


# 5. (Optional) A Small Interactive LLM Playground
This section is optional. You do not need to implement it to complete the project. It is meant purely for exploration and will not significantly affect your core AI engineering skills.

If you are curious, you can build a simple interactive playground to experiment with text generation. You can:
- Create input widgets for the prompt, model selection, decoding strategy, and temperature
- Use Hugging Face's generate method to produce text based on the selected settings
- Display the model's response directly in the notebook output

You may find following links helpful:
- https://ipywidgets.readthedocs.io/en/latest/
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# INTERACTIVE PLAYGROUND: Build your own UI
#
# Concept: Using UI widgets (buttons, text boxes) inside a notebook
#
# Workflow:
#   1. User types in 'Textarea'
#   2. User selects 'Model' and 'Strategy' via 'Dropdowns'
#   3. User clicks 'Generate' button
#   4. 'on_button_clicked' function runs, calls generate(), displays result
#
# Purpose: This brings everything together into a user-friendly interface.
# ═══════════════════════════════════════════════════════════════════

import ipywidgets as widgets
from IPython.display import display, Markdown

# Function called when the 'Generate' button is clicked
def on_button_clicked(b):
    # 1. Determine which model/tokenizer to use
    model = gpt2_model if model_select.value == "GPT-2" else qwen_model
    tokenizer = gpt2_tokenizer if model_select.value == "GPT-2" else qwen_tokenizer
    
    # 2. Clear old output
    output.clear_output()
    
    # 3. Generate and display new text
    with output:
        # Lowercase the strategy (e.g. "Greedy" -> "greedy") for our function
        res = generate(model, tokenizer, prompt_text.value, strategy_select.value.lower().replace('-','_'), 128)
        display(Markdown(res))

# Create the UI components
prompt_text = widgets.Textarea(value="Once upon a time", description="Prompt:")
model_select = widgets.Dropdown(options=["GPT-2", "Qwen"], value="GPT-2", description="Model:")
strategy_select = widgets.Dropdown(options=["Greedy", "Top-k", "Top-p"], value="Greedy", description="Strategy:")
button = widgets.Button(description="Generate")

# Connect the button click to our function
button.on_click(on_button_clicked)
output = widgets.Output()

# Layout the components vertically (VBox)
display(widgets.VBox([prompt_text, model_select, strategy_select, button, output]))



## 🎉 Congratulations!

You've just learned, explored, and inspected a real **LLM**. In one project you:
* Learned how **tokenization** works in practice
* Used `tiktoken` library to load and experiment with most advanced tokenizers.
* Explored LLM architecture and inspected GPT2 blocks and layers
* Learned decoding strategies and used `top-p` to generate text from GPT2
* Loaded a powerful chat model, `Qwen3-0.6B` and generated text
* Built an LLM playground


👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work. The skills you used here power most LLMs you see everywhere.
